# Setting up connection to database

In [2]:
import pyodbc
server = 'DIST-6-505.uopnet.plymouth.ac.uk'
database = 'COMP2001_MFerguson'
username = 'MFerguson'
password = 'GjiF140*'
driver = '{ODBC Driver 17 for SQL Server}'

conn_str = (
    f'DRIVER={driver};'
    f'SERVER={server};'
    f'DATABASE={database};'
    f'UID={username};'
    f'PWD={password};'
    'Encrypt=Yes;'
    'TrustServerCertificate=Yes;'
    'Connection Timeout=30;'
    'Trusted_Connection=No;'
)
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

# Creating schema

In [ ]:
cursor.execute(''' CREATE SCHEMA CW2''')
conn.commit()

# Creating the tables

In [4]:
cursor.execute('''
CREATE TABLE CW2.[User] (
    user_id INT IDENTITY (1,1) PRIMARY KEY,
    username VARCHAR(255) NOT NULL,
    email VARCHAR(255) NOT NULL,
    [password] VARCHAR(255) NOT NULL,
    role VARCHAR(8) NOT NULL,
               
    CHECK (LEN(username) >= 5 AND LEN(username) <= 255),
    CHECK (email LIKE '%_@_%.__%'),
    CHECK (LEN([password]) >= 8 AND CHARINDEX(' ', [password]) = 0),
    CHECK (role = 'admin' OR role = 'user')
)
''')
conn.commit()

In [5]:
cursor.execute('''
INSERT INTO CW2.[User] (username, email, [password], role)
VALUES
('Admin','admin@gmail.com','AdM1NpQsW0Rd','admin'),
('Tim Berners-Lee','tim@plymouth.ac.uk','COMP2001!', 'user'),
('Grace Hopper','grace@plymouth.ac.uk','ISAD123!', 'user'),
('Ada Lovelace','ada@plymouth.ac.uk','insecurePassword','user');
''')
conn.commit()

In [6]:
cursor.execute('''
CREATE TABLE CW2.Location_Point (
    location_point_id INT IDENTITY(1,1) PRIMARY KEY,
    latitude DECIMAL(8,6) NOT NULL,
    longitude DECIMAL(9,6) NOT NULL,
    description VARCHAR(255),  
                   
    CHECK (latitude BETWEEN -90 AND 90),
    CHECK (longitude BETWEEN -180 AND 180),    
    CHECK (LEN(description) >3)       
)
''')
conn.commit()

In [7]:
cursor.execute('''
INSERT INTO CW2.Location_Point (latitude, longitude, description)
VALUES
(50.123456, -4.234567, 'Scenic viewpoint on the hill'),
(51.654321, -3.876543, 'Lake surrounded by forest'),
(52.987654, -2.123456, 'Historic site with ruins'),
(53.345678, -1.654321, 'Trailhead with parking area'),
(54.876543, -0.987654, 'Picnic area with tables and benches');
''')
conn.commit()


In [8]:
cursor.execute('''
CREATE TABLE CW2.Trail (
    trail_id INT IDENTITY(1,1) PRIMARY KEY, 
    trail_name VARCHAR(255) NOT NULL UNIQUE,
    summary VARCHAR(255),
    description VARCHAR(1500),
    location VARCHAR(255) NOT NULL,
    traffic VARCHAR(8) NOT NULL,
    difficulty VARCHAR(8) NOT NULL,
    length DECIMAL(7,2) NOT NULL,           
    duration VARCHAR(5) NOT NULL,
    elevation_gain INT NOT NULL,
    route_type VARCHAR(14) NOT NULL,
               
    owner_id INT NOT NULL, 

    CHECK (LEN(trail_name) >= 5 AND LEN(trail_name) <= 50),
    CHECK (LEN(summary) >= 10 OR LEN(summary) = 0),
    CHECK (LEN(description) >= 50 OR LEN(description) = 0),
    CHECK (lower(traffic)= 'heavy' OR lower(traffic) = 'moderate' OR lower(traffic) = 'light'),
    CHECK (lower(difficulty) = 'hard' OR lower(difficulty) = 'moderate' OR lower(difficulty) = 'easy'),  
    CHECK (length > 0 AND length = ROUND(length, 2)),
    CHECK (duration LIKE'[0-9][0-9]:[0-9][0-9]'),
    CHECK (elevation_gain >=0),
    CHECK (lower(route_type) = 'out and back' OR lower(route_type) = 'loop' OR lower(route_type) = 'point to point'),
               
    CONSTRAINT FK_owner_id FOREIGN KEY (owner_id) REFERENCES CW2.[User](user_id),
)
''')
conn.commit()


In [9]:
cursor.execute('''
INSERT INTO CW2.Trail (trail_name, summary, description, location, traffic,difficulty, length, duration, elevation_gain, route_type, owner_id) 
VALUES
('Forest Walk', 'A scenic walk through dense forest',
    'This trail takes you through a beautiful forest with diverse wildlife and flora.', 'Plymouth, Devon, UK', 'moderate',
    'easy', 5.50, '02:30', 300, 'loop', 2),
('Mountain Hike', 'A challenging hike with rewarding views',
    'A strenuous but rewarding trail up the mountain with breathtaking views at the summit.', 'Los Angeles, California, USA', 'heavy',
    'hard', 12.30, '05:45', 1200, 'out and back', 3),
('Coastal Path', 'A leisurely walk along the coast',
    'This trail offers stunning views of the ocean and a chance to visit sandy beaches.', 'Lisbon, Portugal', 'light',
    'moderate', 8.70, '03:15', 150, 'point to point', 4),
('River Valley Trek', 'A trek along a scenic river valley',
    'Follow the river through a picturesque valley with plenty of photo opportunities.', 'Moscow, Russia', 'moderate',
    'easy', 10.20, '04:00', 400, 'loop', 2);

''')
conn.commit()

In [10]:
cursor.execute('''CREATE TABLE CW2.Trail_LocationPt (
    trail_id INT NOT NULL,
    location_point_id INT NOT NULL,
    order_number INT NOT NULL,
    CONSTRAINT fk_trail_id FOREIGN KEY (trail_id) REFERENCES CW2.Trail(trail_id),
    CONSTRAINT fk_location_point_id FOREIGN KEY (location_point_id) REFERENCES CW2.Location_Point(location_point_id)
)
''')
conn.commit()

In [11]:
cursor.execute('''
INSERT INTO CW2.Trail_LocationPt (trail_id, location_point_id, order_number)
VALUES
(1, 1, 1),
(1, 2, 2),
(1, 3, 3),
(1, 4, 4),
(1, 5, 5),
(2, 2, 1),
(2, 4, 2),
(2, 1, 3),
(2, 2, 4),
(2, 3, 5),
(3, 3, 1),
(3, 4, 2),
(3, 1, 3),
(3, 2, 4),
(3, 3, 5),
(4, 4, 1),
(4, 1, 2),
(4, 2, 3),
(4, 3, 4),
(4, 4, 5);
''')
conn.commit()

In [12]:
cursor.execute('''
CREATE TABLE CW2.Attraction (
    attraction_id INT IDENTITY(1,1) PRIMARY KEY,
    attraction_name VARCHAR(255) NOT NULL,
    CHECK (LEN(attraction_name) >= 3)               
)
''')
conn.commit()

In [13]:
cursor.execute('''
INSERT INTO CW2.Attraction (attraction_name)
VALUES
('Waterfall'),
('Viewpoint'),
('Historic Ruins'),
('Picnic Spot');
''')
conn.commit()


In [14]:
cursor.execute('''
CREATE TABLE CW2.Trail_Attraction (
    trail_id INT NOT NULL,
    attraction_id INT NOT NULL,
    PRIMARY KEY (trail_id, attraction_id),
    
    CONSTRAINT FK_trail_id_attraction FOREIGN KEY (trail_id) REFERENCES CW2.Trail(trail_id),
    CONSTRAINT FK_attraction_id FOREIGN KEY (attraction_id) REFERENCES CW2.Attraction(attraction_id)
)
''')
conn.commit()

In [15]:
cursor.execute('''
INSERT INTO CW2.Trail_Attraction (trail_id, attraction_id)
VALUES
(1, 1),
(2, 2),
(3, 3),
(4, 4),
(1, 2),
(2, 4),
(3, 1);
''')
conn.commit()



# Viewing all the data stored in the tables

In [3]:
cursor.execute(''' SELECT * FROM CW2.[User]''')
row = cursor.fetchall()
for rows in row:
    print(rows)

(1, 'Admin', 'admin@gmail.com', 'AdM1NpQsW0Rd', 'admin')
(2, 'Tim Berners-Lee', 'tim@plymouth.ac.uk', 'COMP2001!', 'user')
(3, 'Grace Hopper', 'grace@plymouth.ac.uk', 'ISAD123!', 'user')
(4, 'Ada Lovelace', 'ada@plymouth.ac.uk', 'insecurePassword', 'user')


In [4]:
cursor.execute(''' SELECT * FROM CW2.Location_Point''')
row = cursor.fetchall()
for rows in row:
    print(rows)

(1, Decimal('50.123456'), Decimal('-4.234567'), 'Scenic viewpoint on the hill')
(2, Decimal('51.654321'), Decimal('-3.876543'), 'Lake surrounded by forest')
(3, Decimal('52.987654'), Decimal('-2.123456'), 'Historic site with ruins')
(4, Decimal('53.345678'), Decimal('-1.654321'), 'Trailhead with parking area')
(5, Decimal('54.876543'), Decimal('-0.987654'), 'Picnic area with tables and benches')


In [5]:
cursor.execute(''' SELECT * FROM CW2.Trail''')
row = cursor.fetchall()
for rows in row:
    print(rows)

(1, 'Forest Walk', 'A scenic walk through dense forest', 'This trail takes you through a beautiful forest with diverse wildlife and flora.', 'Plymouth, Devon, UK', 'moderate', 'easy', Decimal('5.50'), '02:30', 300, 'loop', 2)
(2, 'Mountain Hike', 'A challenging hike with rewarding views', 'A strenuous but rewarding trail up the mountain with breathtaking views at the summit.', 'Los Angeles, California, USA', 'heavy', 'hard', Decimal('12.30'), '05:45', 1200, 'out and back', 3)
(3, 'Coastal Path', 'A leisurely walk along the coast', 'This trail offers stunning views of the ocean and a chance to visit sandy beaches.', 'Lisbon, Portugal', 'light', 'moderate', Decimal('8.70'), '03:15', 150, 'point to point', 4)
(4, 'River Valley Trek', 'A trek along a scenic river valley', 'Follow the river through a picturesque valley with plenty of photo opportunities.', 'Moscow, Russia', 'moderate', 'easy', Decimal('10.20'), '04:00', 400, 'loop', 2)


In [6]:
cursor.execute(''' SELECT * FROM CW2.Attraction''')
row = cursor.fetchall()
for rows in row:
    print(rows)

(1, 'Waterfall')
(2, 'Viewpoint')
(3, 'Historic Ruins')
(4, 'Picnic Spot')


In [7]:
cursor.execute(''' SELECT * FROM CW2.Trail_Attraction''')
row = cursor.fetchall()
for rows in row:
    print(rows)
    

(1, 1)
(1, 2)
(2, 2)
(2, 4)
(3, 1)
(3, 3)
(4, 4)


In [8]:
cursor.execute(''' SELECT * FROM CW2.Trail_LocationPt''')
row = cursor.fetchall()
for rows in row:
    print(rows)
    

(1, 1, 1)
(1, 2, 2)
(1, 3, 3)
(1, 4, 4)
(1, 5, 5)
(2, 2, 1)
(2, 4, 2)
(2, 1, 3)
(2, 2, 4)
(2, 3, 5)
(3, 3, 1)
(3, 4, 2)
(3, 1, 3)
(3, 2, 4)
(3, 3, 5)
(4, 4, 1)
(4, 1, 2)
(4, 2, 3)
(4, 3, 4)
(4, 4, 5)


# Drop all tables

In [3]:
cursor.execute('''
DROP TABLE CW2.Trail_Attraction
DROP TABLE CW2.Trail_LocationPt 
DROP TABLE CW2.Trail
DROP TABLE CW2.Location_Point
DROP TABLE CW2.Attraction
DROP TABLE CW2.[User]
''')
conn.commit()

# A view to trail and attractions

In [30]:
cursor.execute('''

CREATE VIEW CW2.TrailView AS
SELECT 
    Trail.trail_id,
    Trail.trail_name,
    Trail.summary,
    Trail.description,
    Trail.location,
    Trail.traffic,
    Trail.difficulty,
    Trail.length,
    Trail.duration,
    Trail.elevation_gain,
    Trail.route_type,
    Trail.owner_id,
    STRING_AGG
    (
        CONCAT
        (
            'Latitude: ', Location_Point.latitude, 
            ', Longitude: ', Location_Point.longitude
        ), '; '
    ) 
    AS location_points,
    STRING_AGG(Attraction.attraction_name, ', ') AS attractions
FROM 
    CW2.Trail
LEFT JOIN 
    CW2.Trail_LocationPt ON CW2.Trail.trail_id = CW2.Trail_LocationPt.trail_id
LEFT JOIN 
    CW2.Location_Point ON CW2.Trail_LocationPt.location_point_id = CW2.Location_Point.location_point_id
LEFT JOIN 
    CW2.Trail_Attraction ON CW2.Trail.trail_id = CW2.Trail_Attraction.trail_id
LEFT JOIN 
    CW2.Attraction ON CW2.Trail_Attraction.attraction_id = CW2.Attraction.attraction_id
GROUP BY 
    Trail.trail_id,
    Trail.trail_name,
    Trail.summary,
    Trail.description,
    Trail.location,
    Trail.traffic,
    Trail.difficulty,
    Trail.length,
    Trail.duration,
    Trail.elevation_gain,
    Trail.route_type,
    Trail.owner_id;
''')
conn.commit()


# Dropping the view

In [28]:
cursor.execute('''
DROP VIEW CW2.TrailView
''')
conn.commit()

# Stored Procedures for CRUD for each table

In [103]:
cursor.execute('''
CREATE PROCEDURE CW2.CreateTrail (@owner_id as INT, @trail_name as VARCHAR(255), @summary as VARCHAR(255),  
@description as VARCHAR(1500), @location as VARCHAR(255), @traffic as VARCHAR(8), @difficulty as VARCHAR(8), 
@length as DECIMAL(7,2), @duration as VARCHAR(5), @elevation_gain as INT, @route_type as VARCHAR(14))
AS
BEGIN   
    INSERT INTO CW2.Trail ( owner_id, trail_name, summary, description, location, traffic, difficulty, length, duration, 
    elevation_gain, route_type)
    VALUES 
    (@owner_id, @trail_name, @summary, @description, @location, @traffic, @difficulty, @length, @duration,
    @elevation_gain, @route_type);
END;
''')
conn.commit()

In [104]:
cursor.execute('''
CREATE PROCEDURE CW2.RetrieveTrail (@trail_id INT = NULL) AS
BEGIN
    IF @trail_id IS NULL
    BEGIN
        SELECT * FROM CW2.TrailView;
    END
    ELSE
    BEGIN
        SELECT * FROM CW2.TrailView
        WHERE trail_id = @trail_id;
    END
END
''')
conn.commit()

In [105]:
cursor.execute('''
CREATE PROCEDURE CW2.RetrieveTrailByOwner (@owner_id INT) AS
BEGIN
    SELECT * FROM CW2.TrailView
    WHERE owner_id = @owner_id;
END
''')
conn.commit()

In [106]:
cursor.execute('''
Create PROCEDURE CW2.RetrieveTrailByLocation (@location as VARCHAR(255)) AS
BEGIN
    SELECT * FROM CW2.TrailView
    WHERE location = @location;
END
''')
conn.commit()

In [107]:
cursor.execute('''
CREATE PROCEDURE CW2.RetrieveTrailByAttraction (@attraction_id INT) AS
BEGIN
    SELECT * FROM CW2.TrailView
    WHERE trail_id IN 
    (
        SELECT trail_id FROM CW2.Trail_Attraction
        WHERE attraction_id = @attraction_id
    )
END
''')
conn.commit()

In [108]:
cursor.execute('''
CREATE PROCEDURE CW2.UpdateTrail (@trail_id INT = NULL, @owner_id INT = NULL, @trail_name VARCHAR(255) = NULL, @summary VARCHAR(255) = NULL,  @description VARCHAR(1500) = NULL, 
@location VARCHAR(255) = NULL, @traffic VARCHAR(8) = NULL, @difficulty VARCHAR(8) = NULL,  @length DECIMAL(7,2) = NULL, @duration VARCHAR(5) = NULL, @elevation_gain INT = NULL, 
@route_type VARCHAR(14) = NULL)
AS
BEGIN
    BEGIN TRANSACTION;

    BEGIN TRY
        IF NOT EXISTS (SELECT 1 FROM CW2.Trail WHERE trail_id = @trail_id)
        BEGIN
            RAISERROR('Trail ID does not exist.', 16, 1);
            ROLLBACK TRANSACTION;
            RETURN;
        END;
        UPDATE CW2.Trail
        SET
            owner_id = ISNULL(@owner_id, owner_id),
            trail_name = ISNULL(@trail_name, trail_name),
            summary = ISNULL(@summary, summary),
            description = ISNULL(@description, description),
            location = ISNULL(@location, location),
            traffic = ISNULL(@traffic, traffic),
            difficulty = ISNULL(@difficulty, difficulty),
            length = ISNULL(@length, length),
            duration = ISNULL(@duration, duration),
            elevation_gain = ISNULL(@elevation_gain, elevation_gain),
            route_type = ISNULL(@route_type, route_type)
        WHERE trail_id = @trail_id;

        COMMIT TRANSACTION;
    END TRY
    BEGIN CATCH
        ROLLBACK TRANSACTION;
        THROW;
    END CATCH;
END;
''')
conn.commit()

In [109]:
cursor.execute('''
CREATE PROCEDURE CW2.DeleteTrail(@trail_id as INT) AS
BEGIN
    DELETE FROM CW2.Trail_Attraction
    WHERE trail_id = @trail_id;
    DELETE FROM CW2.Trail
    WHERE trail_id = @trail_id
END
''')
conn.commit()

In [110]:
cursor.execute('''
CREATE PROCEDURE CW2.CreateUser (@username as VARCHAR(255), @email as VARCHAR(255), @password as VARCHAR(255), @role as VARCHAR(8))
AS
BEGIN
    INSERT INTO CW2.[User] (username, email, [password], role)
    VALUES (@username, @email, @password, @role);
END
''')
conn.commit()

In [111]:
cursor.execute('''
CREATE PROCEDURE CW2.RetrieveUser (@user_id INT = NULL) AS
BEGIN
    IF @user_id IS NULL
    BEGIN
        SELECT * FROM CW2.[User];
    END
    ELSE
    BEGIN
        SELECT * FROM CW2.[User]
        WHERE user_id = @user_id;
    END
END
''')
conn.commit()

In [112]:
cursor.execute('''
CREATE PROCEDURE CW2.UpdateUser (@user_id as INT = NULL, @username as VARCHAR(255) = NULL, @email as VARCHAR(255) = NULL, @password as VARCHAR(255) = NULL, @role as VARCHAR(8) = NULL)
AS
BEGIN
    BEGIN TRANSACTION;

    BEGIN TRY
        IF NOT EXISTS (SELECT 1 FROM CW2.[User] WHERE user_id = @user_id)
        BEGIN
            RAISERROR('User ID does not exist.', 16, 1);
            ROLLBACK TRANSACTION;
            RETURN;
        END;
        UPDATE CW2.[User]
        SET
            username = ISNULL(@username, username),
            email = ISNULL(@email, email),
            [password] = ISNULL(@password, [password]),
            role = ISNULL(@role, role)
        WHERE user_id = @user_id;

        COMMIT TRANSACTION;
    END TRY
    BEGIN CATCH
        ROLLBACK TRANSACTION;
        THROW;
    END CATCH
END;
''')
conn.commit()

In [113]:
cursor.execute('''
CREATE PROCEDURE CW2.DeleteUser(@user_id as INT) AS
BEGIN
    DELETE FROM CW2.Trail
    WHERE owner_id = @user_id
    DELETE FROM CW2.[User]
    WHERE user_id = @user_id
END
''')
conn.commit()

In [114]:
cursor.execute('''
CREATE PROCEDURE CW2.CreateLocationPoint (@latitude as DECIMAL(9,6), @longitude as DECIMAL(9,6), @description as VARCHAR(255))
AS
BEGIN
    INSERT INTO CW2.Location_Point  (latitude, longitude, description)
    VALUES (@latitude, @longitude, @description);
END
''')
conn.commit()

In [115]:
cursor.execute('''
CREATE PROCEDURE CW2.RetrieveLocationPoint (@location_point_id INT = NULL) AS
BEGIN
    IF @location_point_id IS NULL
    BEGIN
        SELECT * FROM CW2.Location_Point;
    END
    ELSE
    BEGIN
        SELECT * FROM CW2.Location_Point
        WHERE location_point_id = @location_point_id;
    END
END
''')
conn.commit()

In [116]:
cursor.execute('''
CREATE PROCEDURE CW2.UpdateLocationPoint (@location_point_id as INT = NULL, @latitude as DECIMAL(9,6) = NULL, @longitude as DECIMAL(9,6) = NULL, @description as VARCHAR(255) = NULL)
AS
BEGIN
    BEGIN TRANSACTION;

    BEGIN TRY
        IF NOT EXISTS (SELECT 1 FROM CW2.Location_Point WHERE location_point_id = @location_point_id)
        BEGIN
            RAISERROR('Location Point ID does not exist.', 16, 1);
            ROLLBACK TRANSACTION;
            RETURN;
        END;
        UPDATE CW2.Location_Point
        SET
            latitude = ISNULL(@latitude, latitude),
            longitude = ISNULL(@longitude, longitude),
            description = ISNULL(@description, description)
        WHERE location_point_id = @location_point_id;

        COMMIT TRANSACTION;
    END TRY
    BEGIN CATCH
        ROLLBACK TRANSACTION;
        THROW;
    END CATCH
END;
''')
conn.commit()

In [117]:
cursor.execute('''
CREATE PROCEDURE CW2.DeleteLocationPoint(@location_point_id INT) AS
BEGIN
    DELETE FROM CW2.TrailLocationPt
    WHERE location_point_id = @location_point_id;

    DELETE FROM CW2.Location_Point
    WHERE location_point_id = @location_point_id;
END
''')
conn.commit()

In [118]:
cursor.execute('''
CREATE PROCEDURE CW2.CreateAttraction (@attraction_name as VARCHAR(255))
AS
BEGIN
    INSERT INTO CW2.Attraction (attraction_name)
    VALUES (@attraction_name);
END
''')
conn.commit()

In [119]:
cursor.execute('''
CREATE PROCEDURE CW2.RetrieveAttraction (@attraction_id INT = NULL) AS
BEGIN
    IF @attraction_id IS NULL
    BEGIN
        SELECT * FROM CW2.Attraction;
    END
    ELSE
    BEGIN
        SELECT * FROM CW2.Attraction
        WHERE attraction_id = @attraction_id;
    END
END
''')
conn.commit()

In [120]:
cursor.execute('''
CREATE PROCEDURE CW2.UpdateAttraction (@attraction_id as INT = NULL, @attraction_name as VARCHAR(255) = NULL)
AS
BEGIN
    BEGIN TRANSACTION;

    BEGIN TRY
        IF NOT EXISTS (SELECT 1 FROM CW2.Attraction WHERE attraction_id = @attraction_id)
        BEGIN
            RAISERROR('Attraction ID does not exist.', 16, 1);
            ROLLBACK TRANSACTION;
            RETURN;
        END;
        UPDATE CW2.Attraction
        SET
            attraction_name = ISNULL(@attraction_name, attraction_name)
        WHERE attraction_id = @attraction_id;

        COMMIT TRANSACTION;
    END TRY
    BEGIN CATCH
        ROLLBACK TRANSACTION;
        THROW;
    END CATCH
END;
''')
conn.commit()

In [121]:
cursor.execute('''
CREATE PROCEDURE CW2.DeleteAttraction(@attraction_id as INT) AS
BEGIN
    DELETE FROM CW2.Trail_Attraction
    WHERE attraction_id = @attraction_id
    DELETE FROM CW2.Attraction
    WHERE attraction_id = @attraction_id
END
''')
conn.commit()

In [122]:
cursor.execute('''
CREATE PROCEDURE CW2.CreateTrailAttraction (@trail_id as INT, @attraction_id as INT)
AS
BEGIN
    INSERT INTO CW2.Trail_Attraction (trail_id, attraction_id)
    VALUES (@trail_id, @attraction_id);
END
''')
conn.commit()

In [123]:
cursor.execute('''
CREATE PROCEDURE CW2.RetrieveTrailAttraction (@trail_id INT = NULL, @attraction_id INT = NULL) AS
BEGIN
    IF @trail_id IS NULL AND @attraction_id IS NULL
    BEGIN
        SELECT * FROM CW2.Trail_Attraction;
    END
    ELSE IF @trail_id IS NOT NULL AND @attraction_id IS NULL
    BEGIN
        SELECT * FROM CW2.Trail_Attraction
        WHERE trail_id = @trail_id;
    END
    ELSE IF @trail_id IS NULL AND @attraction_id IS NOT NULL
    BEGIN
        SELECT * FROM CW2.Trail_Attraction
        WHERE attraction_id = @attraction_id;
    END
    ELSE
    BEGIN
        SELECT * FROM CW2.Trail_Attraction
        WHERE trail_id = @trail_id AND attraction_id = @attraction_id;
    END
END
''')
conn.commit()

In [124]:
cursor.execute('''
CREATE PROCEDURE CW2.DeleteTrailAttraction(@trail_id as INT, @attraction_id as INT) AS
BEGIN
    DELETE FROM CW2.Trail_Attraction
    WHERE trail_id = @trail_id AND attraction_id = @attraction_id
END
''')
conn.commit()

In [125]:
cursor.execute('''
CREATE PROCEDURE CW2.CreateTrailLocationPt (@trail_id as INT, @location_point_id as INT, @order_number as INT)
AS
BEGIN
    INSERT INTO CW2.Trail_LocationPt (trail_id, location_point_id, order_number)
    VALUES (@trail_id, @location_point_id, @order_number);
END
               
''')
conn.commit()

In [126]:
cursor.execute('''
CREATE PROCEDURE CW2.RetrieveTrailLocationPt (@trail_id INT = NULL, @location_point_id INT = NULL) AS
BEGIN
    IF @trail_id IS NULL AND @location_point_id IS NULL
    BEGIN
        SELECT * FROM CW2.Trail_LocationPt;
    END
    ELSE IF @trail_id IS NOT NULL AND @location_point_id IS NULL
    BEGIN
        SELECT * FROM CW2.Trail_LocationPt
        WHERE trail_id = @trail_id;
    END
    ELSE IF @trail_id IS NULL AND @location_point_id IS NOT NULL
    BEGIN
        SELECT * FROM CW2.Trail_LocationPt
        WHERE location_point_id = @location_point_id;
    END
    ELSE
    BEGIN
        SELECT * FROM CW2.Trail_LocationPt
        WHERE trail_id = @trail_id AND location_point_id = @location_point_id;
    END
END
''')
conn.commit()

In [127]:
cursor.execute(''' 
CREATE PROCEDURE CW2.UpdateTrailLocationPt (@trail_id as INT, @location_point_id as INT, @order_number as INT)
AS
BEGIN
    BEGIN TRANSACTION;

    BEGIN TRY
        IF NOT EXISTS (SELECT 1 FROM CW2.Trail_LocationPt WHERE trail_id = @trail_id AND location_point_id = @location_point_id)
        BEGIN
            RAISERROR('Trail ID and Location Point ID combination does not exist.', 16, 1);
            ROLLBACK TRANSACTION;
            RETURN;
        END;
        UPDATE CW2.Trail_LocationPt
        SET
            order_number = @order_number
        WHERE trail_id = @trail_id AND location_point_id = @location_point_id;

        COMMIT TRANSACTION;
    END TRY
    BEGIN CATCH
        ROLLBACK TRANSACTION;
        THROW;
    END CATCH
END;
''')
conn.commit()

In [128]:
cursor.execute('''
CREATE PROCEDURE CW2.DeleteTrailLocationPt(@trail_id as INT, @location_point_id as INT) AS
BEGIN
    DELETE FROM CW2.Trail_LocationPt
    WHERE trail_id = @trail_id AND location_point_id = @location_point_id
END
''')
conn.commit()

# Dropping all procedures

In [102]:
cursor.execute('''
DROP PROCEDURE CW2.CreateTrail
DROP PROCEDURE CW2.RetrieveTrail
DROP PROCEDURE CW2.RetrieveTrailByOwner
DROP PROCEDURE CW2.RetrieveTrailByLocation
DROP PROCEDURE CW2.RetrieveTrailByAttraction
DROP PROCEDURE CW2.UpdateTrail
DROP PROCEDURE CW2.DeleteTrail
DROP PROCEDURE CW2.CreateUser
DROP PROCEDURE CW2.RetrieveUser
DROP PROCEDURE CW2.UpdateUser
DROP PROCEDURE CW2.DeleteUser
DROP PROCEDURE CW2.CreateLocationPoint
DROP PROCEDURE CW2.RetrieveLocationPoint
DROP PROCEDURE CW2.UpdateLocationPoint
DROP PROCEDURE CW2.DeleteLocationPoint
DROP PROCEDURE CW2.CreateAttraction
DROP PROCEDURE CW2.RetrieveAttraction
DROP PROCEDURE CW2.UpdateAttraction
DROP PROCEDURE CW2.DeleteAttraction
DROP PROCEDURE CW2.CreateTrailAttraction
DROP PROCEDURE CW2.RetrieveTrailAttraction
DROP PROCEDURE CW2.DeleteTrailAttraction
DROP PROCEDURE CW2.CreateTrailLocationPt
DROP PROCEDURE CW2.RetrieveTrailLocationPt
DROP PROCEDURE CW2.UpdateTrailLocationPt
DROP PROCEDURE CW2.DeleteTrailLocationPt
''')
conn.commit()
